<a href="https://colab.research.google.com/github/lwerth/escuelas_crimen_cdmx/blob/master/cdmx_escuelas_merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np, scipy as sp, pymc3 as pm, matplotlib.pyplot as plt, pandas as pd, theano, theano.tensor as tt
from scipy import stats
import seaborn as sb
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
import re
import nltk
from nltk import tokenize
import seaborn as sns
import json

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

import theano

In [43]:
#import fidegar files

fidegar = pd.read_excel('/content/drive/My Drive/Projects/CDMX Schools/fidegar.xlsx')
fidegar_idx = OrdinalEncoder(dtype = np.int64).fit_transform(fidegar[['NIVEL']])

fidegar['nivel'] = fidegar['NIVEL']
fidegar['nombre'] = fidegar['NOMBRE']
fidegar_idx
n_niveles_fidegar = len(fidegar.nivel.unique())
n_niveles_fidegar

fidegar.nivel.unique()
fidegar

,NIVEL,NOMBRE,CCT 1,CCT 2,CCT 3,CCT 4,MATRÍCULA,ALCALDÍA,ESCUELAS EN PROCESO JURÍDICO,nivel,nombre
0,PRIMARIA,JOSE INES NOVELO,09DPR4013T,09DPR0033F,NaN,NaN,646,ALVARO OBREGON,NaN,PRIMARIA,JOSE INES NOVELO
1,PRIMARIA,MAESTRO ENRIQUE COCA PEREZ,09DPR1899N,NaN,NaN,NaN,320,ALVARO OBREGON,NaN,PRIMARIA,MAESTRO ENRIQUE COCA PEREZ
2,PREESCOLAR,RICARDO ZAMORA BUITRON,09DJN0819O,NaN,NaN,NaN,192,ALVARO OBREGON,NaN,PREESCOLAR,RICARDO ZAMORA BUITRON
3,PRIMARIA,EFRAIN HUERTA,09DPR0131G,09DPR4019N,NaN,NaN,2174,ALVARO OBREGON,NaN,PRIMARIA,EFRAIN HUERTA
4,PREESCOLAR,REPUBLICA DE KENIA,09DJN0458U,09DJN0319T,NaN,NaN,441,ALVARO OBREGON,NaN,PREESCOLAR,REPUBLICA DE KENIA
...,...,...,...,...,...,...,...,...,...,...,...
2691,PREESCOLAR,PUNTLI,09DJN1204Z,NaN,NaN,NaN,201,XOCHIMILCO,NaN,PREESCOLAR,PUNTLI
2692,SECUNDARIA,GUILLERMO PRIETO PRADILLO,09DES4295K,09DES0295J,NaN,NaN,1081,XOCHIMILCO,NaN,SECUNDARIA,GUILLERMO PRIETO PRADILLO
2693,PREESCOLAR,PROFR. ISAIAS MORONES ESCAMILLA,09DJN0633J,09DJN1384Z,NaN,NaN,338,XOCHIMILCO,NaN,PREESCOLAR,PROFR. ISAIAS MORONES ESCAMILLA
2694,PRIMARIA,PROFR. CARITINO MALDONADO PEREZ,09DPR0011U,09DPR3193N,NaN,NaN,733,XOCHIMILCO,NaN,PRIMARIA,PROFR. CARITINO MALDONADO PEREZ


In [60]:
# read in and look at gob data

#parse text in delegacion, delete the word "delegacion" in data, alphabetize, create index, and join!

publicas = pd.read_excel('/content/drive/My Drive/Projects/CDMX Schools/escuelas-publicas.xlsx')
pub_idx = OrdinalEncoder(dtype = np.int64).fit_transform(publicas[['nivel']])

n_niveles_pub = len(publicas.nivel.unique())
n_niveles_pub

n_delegacion_pub = len(publicas.Delegacion.unique())


print(n_delegacion_pub, n_niveles_pub)
print('delegacion',publicas.Delegacion.unique())
print('niveles',publicas.nivel.unique())
publicas


36 14
delegacion ['DELEGACION IZTACALCO' 'DELEGACION COYOACAN' 'DELEGACION TLAHUAC'
 'DELEGACION IZTAPALAPA' 'DELEGACION VENUSTIANO CARRANZA'
 'DELEGACION CUAUHTEMOC' 'DELEGACION TLALPAN' 'DELEGACION MILPA ALTA'
 'DELEGACION MIGUEL HIDALGO' 'DELEGACION ALVARO OBREGON'
 'DELEGACION CUAJIMALPA DE MORELOS' 'DELEGACION XOCHIMILCO' nan
 'DELEGACION GUSTAVO A. MADERO' 'DELEGACION BENITO JUAREZ'
 'BATALLONES ROJOS NO 205 LA ALBARRADA' 'AZCAPOTZALCO'
 'SALVADOR DIAZ MIRON' 'SELENE' 'ALVARO OBREGON'
 'AV DEL TRABAJO NO 258 MORELOS' 'DELEGACION ALVARO OBREG├ôN' 'SANTA ROSA'
 'CERRADAS JUAN ESCUTIA' 'MEXICO DF.' 'PRIMARIA GENERAL'
 'DELEGACION AZCAPOTZALCO OBREGON' 'ADOLFO LOPEZ MATEOS' 'EL ROSARIO'
 'TELESECUNDARIA 89' 'EMILIO CARRANZA NO 39 ALBERT.' 'SAN JOSE'
 'SAN JUAN DE ARAGON' '553 Y 504 SECC. VECINAL DOS' '-UNIDAD HA'
 'SANTA CRUZ MEYEHUALCO']
niveles ['INTERVENCION' 'PRIMARIA' 'PREESCOLAR' 'ESCUELA' 'SECUNDARIA'
 'TELESECUNDARIA' 'LACTANTE,' 'SEC.' 'BRINDAR' 'SUPERVISAR' 'ATENDER'
 'LICE

,nivel,nivel_2,nivel_3,nombre,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13
0,INTERVENCION,"TEMPRANA,","PREESCOLAR,",CENTRO DE ATENCION MULTIPLE 82,DOMICILIO: AV TEZONTLE S/N ESQ ANDADOR AGAV...,"-99.10185339944889,19.38575798109795,0",19.385758,-99.101853,AV TEZONTLE S/N ESQ ANDADOR AGAVE,UNIDAD INFONAVIT IZTACALCO,DELEGACION IZTACALCO,MEXICO DF.,NaN,NaN
1,PRIMARIA,GENERAL,NaN,PROFRA. MARIA EPIGMENIA ARRIAGA SALGADO,"DOMICILIO: 2O RET ROSARIO CASTELLANOS MZA L, U...","-99.10427446479174,19.30970837543437,0",19.309708,-99.104274,2O RET ROSARIO CASTELLANOS MZA L,UNIDAD HABITACIONAL C T M,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN
2,PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN
3,PREESCOLAR,GENERAL,CON,EJERCITO DE ORIENTE,"DOMICILIO: RETORNO 2 Y RIFLEROS SLP NO 18, U H...","-99.0368889998984,19.38020299985893,0",19.380203,-99.036889,RETORNO 2 Y RIFLEROS SLP NO 18,U HAB EJERCITO DE OTE II ISSSTE,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
4,PRIMARIA,GENERAL,NaN,ENRIQUE GARCIA GALLEGOS,"DOMICILIO: LATERAL RIO CHURUBUSCO NO 2, SAN J...","-99.11412044387679,19.3699792020639,0",19.369979,-99.114120,LATERAL RIO CHURUBUSCO NO 2,SAN JOSE ACULCO,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,PRIMARIA,GENERAL,NaN,IGNACIO MARISCAL,"DOMICILIO: AGRUPAMIENTO F 101, U SAN JUAN DE A...","-99.056418000091,19.45569799983959,0",19.455698,-99.056418,AGRUPAMIENTO F 101,U SAN JUAN DE ARAGON,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN
2379,SECUNDARIA,GENERAL,NaN,MIGUEL DE CERVANTES SAAVEDRA,"ANAXAGORAS NO 324, NARVARTE.","-99.1568090002494,19.39651199998512,0",19.396512,-99.156809,NaN,NaN,NaN,NaN,NaN,NaN
2380,PREESCOLAR,GENERAL,NaN,IRENE MARIN DE AYALA,"DOMICILIO: HUICHAPAN S/N, MICHOACANA, DELEGACI...","-99.11172663949883,19.44757589685136,0",19.447576,-99.111727,HUICHAPAN S/N,MICHOACANA,DELEGACION VENUSTIANO CARRANZA,MEXICO DF.,NaN,NaN
2381,SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN


In [64]:
niveles_conjun = pd.merge(publicas, fidegar)
# niveles_conjun

## Index the nombres of each dataset

#### Merege and drop duplicates
Here we end up multiplying the datasets to get 3049 rows, which isn't bad. I dropped the duplicates and still got 3049 rows, this seems promising. The original fidegar has 2696 and the origianl escuelas publicas has 2383, i.e. fidegar has ~300 more schools listed.

> *I'm going to keep this set without duplicates and seet how that helps us.*

In [67]:
#cerate index of nombre and merge there
pub_nombre_idx = publicas.set_index('nombre')
fide_nombre_idx = fidegar.set_index('nombre')
# display(pub_nivel_idx, fide_nivel_idx)

conjun_nombre_idx = pd.merge(pub_nombre_idx, fide_nombre_idx,
                      left_index = True, right_index = True)
conjun_nombre_idx

,nivel_x,nivel_2,nivel_3,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13,NIVEL,NOMBRE,CCT 1,CCT 2,CCT 3,CCT 4,MATRÍCULA,ALCALDÍA,ESCUELAS EN PROCESO JURÍDICO,nivel_y
nombre,,,,,,,,,,,,,,,,,,,,,,,
10 DE ABRIL,PREESCOLAR,GENERAL,NaN,"DOMICILIO: CERRADA LEANDRO VALLE S/N, SAN LORE...","-99.32721599994483,19.32895800024563,0",19.328958,-99.327216,CERRADA LEANDRO VALLE S/N,SAN LORENZO ACOPILCO,DELEGACION CUAJIMALPA DE MORELOS,MEXICO DF.,NaN,NaN,PREESCOLAR,10 DE ABRIL,09DJN1096H,09DJN0615U,NaN,NaN,442,CUAJIMALPA DE MORELOS,NaN,PREESCOLAR
14 DE JULIO,PRIMARIA,GENERAL,NaN,"DOMICILIO: PRIV JORULLO S/N, DEL GAS, DELEGAC...","-99.15724799985777,19.47008199998913,0",19.470082,-99.157248,PRIV JORULLO S/N,DEL GAS,AZCAPOTZALCO,MEXICO DF.,NaN,NaN,PRIMARIA,14 DE JULIO,09DPR2138N,NaN,NaN,NaN,164,AZCAPOTZALCO,NaN,PRIMARIA
15 DE MAYO,PRIMARIA,GENERAL,NaN,"DOMICILIO: OSA MAYOR Y GEMELOS S/N, PRADO CHUR...","-99.12892898301334,19.35535455043778,0",19.355355,-99.128929,OSA MAYOR Y GEMELOS S/N,PRADO CHURUBUSCO,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN,PRIMARIA,15 DE MAYO,09DPR1189N,NaN,NaN,NaN,353,COYOACAN,NaN,PRIMARIA
15 DE SEPTIEMBRE,PRIMARIA,GENERAL,NaN,"AV CUITLAHUAC Y AV CEYLAN S/N, AMPLIACION COSM...","-99.16121137376389,19.47235025204103,0",19.472350,-99.161211,NaN,NaN,NaN,NaN,NaN,NaN,PRIMARIA,15 DE SEPTIEMBRE,09DPR1580S,NaN,NaN,NaN,175,AZCAPOTZALCO,NaN,PRIMARIA
18 DE MARZO,PREESCOLAR,GENERAL,CON,"DOMICILIO: ESQ. DE HABANA Y MONTIEL, TEPEYAC I...","-99.12238045081928,19.4847393232064,0",19.484739,-99.122380,ESQ. DE HABANA Y MONTIEL,TEPEYAC INSURGENTES,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN,PRIMARIA,18 DE MARZO,09DPR0907Z,NaN,NaN,NaN,176,CUAUHTEMOC,NaN,PRIMARIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YURI A. GAGARIN,SECUNDARIA,GENERAL,NaN,"DOMICILIO: QUERETARO Y GUADALAJARA S/N, MIGUEL...","-99.1841029095791,19.28600840067503,0",19.286008,-99.184103,QUERETARO Y GUADALAJARA S/N,MIGUEL HIDALGO,DELEGACION TLALPAN,MEXICO DF.,NaN,NaN,SECUNDARIA,YURI A. GAGARIN,09DES0173Z,09DES4173Z,NaN,NaN,694,TLALPAN,NaN,SECUNDARIA
ZACATENCO,PREESCOLAR,GENERAL,NaN,"DOMICILIO: AV. ACUEDUCTO NO 807, SAN PEDRO ZAC...","-99.12278299995941,19.50778800012884,0",19.507788,-99.122783,AV. ACUEDUCTO NO 807,SAN PEDRO ZACATENCO,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN,PREESCOLAR,ZACATENCO,09DJN0194B,NaN,NaN,NaN,158,GUSTAVO A. MADERO,NaN,PREESCOLAR
ZAMBIA,PRIMARIA,GENERAL,NaN,"DOMICILIO: MANZANA H NO 24, COLONIA U HAB LOMA...","-99.22793300016258,19.33521099983617,0",19.335211,-99.227933,MANZANA H NO 24,COLONIA U HAB LOMAS DE PLATEROS,DELEGACION ALVARO OBREGON,MEXICO DF,NaN,NaN,PRIMARIA,ZAMBIA,09DPR2687R,09DPR2222L,NaN,NaN,499,ALVARO OBREGON,NaN,PRIMARIA


In [81]:
# conjun_nombre_idx.to_excel(r'C:\Users\LW\Desktop\cdmx-escuelas_merged.xlsx', index = False, header=True)
# conjun_nombre_idx.to_csv(r'C:\Users\LW\Documents\Project Research\cdmx-escuelas_merged.csv', index = False, header=True)

conjun_nombre_idx.to_excel("C:\\Users\\LW\\Documents\\Project Research\\escuelas.xlsx")  

# with pd.ExcelWriter('cdmx-escuelas_merged.xlsx',
#                     mode='a') as writer:  
#     conjun_nombre_idx.to_excel(writer, sheet_name='conjun_nombre_idx')
    
    # conjun_nombre_idx.to_excel('cdmx-escuelas_merged.csv')


In [68]:
# conjun_nombre_idx = conjun_nombre_idx.drop(columns=['NIVEL', 'NOMBRE', 'CCT 1', 'CCT 2', 'CCT 3', 'CCT 4', 'MATRÍCULA', 'ALCALDÍA', 'ESCUELAS EN PROCESO JURÍDICO', 'nombre_y'])
conjun_nombre_idx = conjun_nombre_idx.drop_duplicates()
conjun_nombre_idx

,nivel_x,nivel_2,nivel_3,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13,NIVEL,NOMBRE,CCT 1,CCT 2,CCT 3,CCT 4,MATRÍCULA,ALCALDÍA,ESCUELAS EN PROCESO JURÍDICO,nivel_y
nombre,,,,,,,,,,,,,,,,,,,,,,,
10 DE ABRIL,PREESCOLAR,GENERAL,NaN,"DOMICILIO: CERRADA LEANDRO VALLE S/N, SAN LORE...","-99.32721599994483,19.32895800024563,0",19.328958,-99.327216,CERRADA LEANDRO VALLE S/N,SAN LORENZO ACOPILCO,DELEGACION CUAJIMALPA DE MORELOS,MEXICO DF.,NaN,NaN,PREESCOLAR,10 DE ABRIL,09DJN1096H,09DJN0615U,NaN,NaN,442,CUAJIMALPA DE MORELOS,NaN,PREESCOLAR
14 DE JULIO,PRIMARIA,GENERAL,NaN,"DOMICILIO: PRIV JORULLO S/N, DEL GAS, DELEGAC...","-99.15724799985777,19.47008199998913,0",19.470082,-99.157248,PRIV JORULLO S/N,DEL GAS,AZCAPOTZALCO,MEXICO DF.,NaN,NaN,PRIMARIA,14 DE JULIO,09DPR2138N,NaN,NaN,NaN,164,AZCAPOTZALCO,NaN,PRIMARIA
15 DE MAYO,PRIMARIA,GENERAL,NaN,"DOMICILIO: OSA MAYOR Y GEMELOS S/N, PRADO CHUR...","-99.12892898301334,19.35535455043778,0",19.355355,-99.128929,OSA MAYOR Y GEMELOS S/N,PRADO CHURUBUSCO,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN,PRIMARIA,15 DE MAYO,09DPR1189N,NaN,NaN,NaN,353,COYOACAN,NaN,PRIMARIA
15 DE SEPTIEMBRE,PRIMARIA,GENERAL,NaN,"AV CUITLAHUAC Y AV CEYLAN S/N, AMPLIACION COSM...","-99.16121137376389,19.47235025204103,0",19.472350,-99.161211,NaN,NaN,NaN,NaN,NaN,NaN,PRIMARIA,15 DE SEPTIEMBRE,09DPR1580S,NaN,NaN,NaN,175,AZCAPOTZALCO,NaN,PRIMARIA
18 DE MARZO,PREESCOLAR,GENERAL,CON,"DOMICILIO: ESQ. DE HABANA Y MONTIEL, TEPEYAC I...","-99.12238045081928,19.4847393232064,0",19.484739,-99.122380,ESQ. DE HABANA Y MONTIEL,TEPEYAC INSURGENTES,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN,PRIMARIA,18 DE MARZO,09DPR0907Z,NaN,NaN,NaN,176,CUAUHTEMOC,NaN,PRIMARIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YURI A. GAGARIN,SECUNDARIA,GENERAL,NaN,"DOMICILIO: QUERETARO Y GUADALAJARA S/N, MIGUEL...","-99.1841029095791,19.28600840067503,0",19.286008,-99.184103,QUERETARO Y GUADALAJARA S/N,MIGUEL HIDALGO,DELEGACION TLALPAN,MEXICO DF.,NaN,NaN,SECUNDARIA,YURI A. GAGARIN,09DES0173Z,09DES4173Z,NaN,NaN,694,TLALPAN,NaN,SECUNDARIA
ZACATENCO,PREESCOLAR,GENERAL,NaN,"DOMICILIO: AV. ACUEDUCTO NO 807, SAN PEDRO ZAC...","-99.12278299995941,19.50778800012884,0",19.507788,-99.122783,AV. ACUEDUCTO NO 807,SAN PEDRO ZACATENCO,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN,PREESCOLAR,ZACATENCO,09DJN0194B,NaN,NaN,NaN,158,GUSTAVO A. MADERO,NaN,PREESCOLAR
ZAMBIA,PRIMARIA,GENERAL,NaN,"DOMICILIO: MANZANA H NO 24, COLONIA U HAB LOMA...","-99.22793300016258,19.33521099983617,0",19.335211,-99.227933,MANZANA H NO 24,COLONIA U HAB LOMAS DE PLATEROS,DELEGACION ALVARO OBREGON,MEXICO DF,NaN,NaN,PRIMARIA,ZAMBIA,09DPR2687R,09DPR2222L,NaN,NaN,499,ALVARO OBREGON,NaN,PRIMARIA


## Index the niveles of each dataset

#### Merege and drop duplicates
Here we end up multiplying the datasets to get ~2220000 rows, then I dropped the duplicates and got only 2140 rows. The original fidegar has 2696 and the origianl escuelas publicas has 2383, i.e. fidegar has ~300 more schools listed.

In [54]:
#cerate index of nivel and merge there
pub_nivel_idx = publicas.set_index('nivel')
fide_nivel_idx = fidegar.set_index('nivel')
# display(pub_nivel_idx, fide_nivel_idx)

conjun_idx = pd.merge(pub_nivel_idx, fide_nivel_idx,
                      left_index = True, right_index = True)
conjun_idx


,nivel_2,nivel_3,nombre_x,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13,NIVEL,NOMBRE,CCT 1,CCT 2,CCT 3,CCT 4,MATRÍCULA,ALCALDÍA,ESCUELAS EN PROCESO JURÍDICO,nombre_y
nivel,,,,,,,,,,,,,,,,,,,,,,,
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN,PREESCOLAR,RICARDO ZAMORA BUITRON,09DJN0819O,NaN,NaN,NaN,192,ALVARO OBREGON,NaN,RICARDO ZAMORA BUITRON
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN,PREESCOLAR,REPUBLICA DE KENIA,09DJN0458U,09DJN0319T,NaN,NaN,441,ALVARO OBREGON,NaN,REPUBLICA DE KENIA
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN,PREESCOLAR,MAESTRO JOSE CALVO,09DJN0110D,NaN,NaN,NaN,121,ALVARO OBREGON,NaN,MAESTRO JOSE CALVO
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN,PREESCOLAR,ALFONSO QUIROZ,09DJN0493Z,NaN,NaN,NaN,231,ALVARO OBREGON,NaN,ALFONSO QUIROZ
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN,PREESCOLAR,MA. EVELIA MONTERRUBIO SAENZ,09DJN1160S,NaN,NaN,NaN,143,ALVARO OBREGON,NaN,MA. EVELIA MONTERRUBIO SAENZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN,SECUNDARIA,HUETZALIN,09DES4224Q,09DES0224P,NaN,NaN,937,XOCHIMILCO,NaN,HUETZALIN
SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN,SECUNDARIA,OCELOTL,09DES0290O,09DES4290P,NaN,NaN,741,XOCHIMILCO,NaN,OCELOTL
SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN,SECUNDARIA,TOMAS ALVA EDISON,09DES0161U,NaN,NaN,NaN,800,XOCHIMILCO,NaN,TOMAS ALVA EDISON


In [58]:
conjun_idx = conjun_idx.drop(columns=['NIVEL', 'NOMBRE', 'CCT 1', 'CCT 2', 'CCT 3', 'CCT 4', 'MATRÍCULA', 'ALCALDÍA', 'ESCUELAS EN PROCESO JURÍDICO', 'nombre_y'])
conjun_idx = conjun_idx.drop_duplicates()
conjun_idx

,nivel_2,nivel_3,nombre_x,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13
nivel,,,,,,,,,,,,,
PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN
PREESCOLAR,GENERAL,CON,EJERCITO DE ORIENTE,"DOMICILIO: RETORNO 2 Y RIFLEROS SLP NO 18, U H...","-99.0368889998984,19.38020299985893,0",19.380203,-99.036889,RETORNO 2 Y RIFLEROS SLP NO 18,U HAB EJERCITO DE OTE II ISSSTE,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
PREESCOLAR,GENERAL,CON,SALVADOR NOVO,"DOMICILIO: LA PURISIMA 208, PURISIMA, DELEGACI...","-99.09861799999243,19.37772499996316,0",19.377725,-99.098618,LA PURISIMA 208,PURISIMA,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
PREESCOLAR,GENERAL,NaN,YOALLI,DOMICILIO: AV TENOCHTITLAN Y AV RIO CHURUBUSCO...,"-99.05894341179388,19.42262605219006,0",19.422626,-99.058943,AV TENOCHTITLAN Y AV RIO CHURUBUSCO,ARENAL 1A SECC,DELEGACION VENUSTIANO CARRANZA,MEXICO DF.,NaN,NaN
PREESCOLAR,GENERAL,NaN,NAZUL,"DOMICILIO: JACARANDAS S/N, SANTA MARTHA ACATIT...","-99.01162350787458,19.34585171256843,0",19.345852,-99.011624,JACARANDAS S/N,SANTA MARTHA ACATITLA,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
SECUNDARIA,GENERAL,NaN,JOSE MARIA MORELOS Y PAVON,"DOMICILIO: NUEVO LEON NO 178, HIPODROMO DE LA ...","-99.17184499964395,19.40653100031749,0",19.406531,-99.171845,NUEVO LEON NO 178,HIPODROMO DE LA CONDESA,DELEGACION CUAUHTEMOC,MEXICO DF.,NaN,NaN
SECUNDARIA,GENERAL,NaN,ANTONIO CARRILLO FLORES,"DOMICILIO: CARRETERA TECNOLOGICO CHIMALPA, CHI...","-99.31719400008167,19.34899199979476,0",19.348992,-99.317194,CARRETERA TECNOLOGICO CHIMALPA,CHIMALPA,DELEGACION CUAJIMALPA DE MORELOS,MEXICO DF.,NaN,NaN
SECUNDARIA,GENERAL,NaN,GABRIELA MISTRAL,"DOMICILIO: JOSE ZUBIETA NO 54, JUAN ESCUTIA, D...","-99.0433450000578,19.3921450001754,0",19.392145,-99.043345,JOSE ZUBIETA NO 54,JUAN ESCUTIA,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN


## Drop words to merge easier

here I try to play around with the stings in the columns to see if we could get matches ... it didn't exactly work.

In [ ]:
publicas['Delegacion'] = publicas['Delegacion'].str.strip('DELEGACION')

publicas

,nivel,nivel_2,nivel_3,nombre,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13
0,INTERVENCION,"TEMPRANA,","PREESCOLAR,",CENTRO DE ATENCION MULTIPLE 82,DOMICILIO: AV TEZONTLE S/N ESQ ANDADOR AGAV...,"-99.10185339944889,19.38575798109795,0",19.385758,-99.101853,AV TEZONTLE S/N ESQ ANDADOR AGAVE,UNIDAD INFONAVIT IZTACALCO,IZT,MEXICO DF.,NaN,NaN
1,PRIMARIA,GENERAL,NaN,PROFRA. MARIA EPIGMENIA ARRIAGA SALGADO,"DOMICILIO: 2O RET ROSARIO CASTELLANOS MZA L, U...","-99.10427446479174,19.30970837543437,0",19.309708,-99.104274,2O RET ROSARIO CASTELLANOS MZA L,UNIDAD HABITACIONAL C T M,COY,MEXICO DF.,NaN,NaN
2,PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,TLAHU,MEXICO DF.,NaN,NaN
3,PREESCOLAR,GENERAL,CON,EJERCITO DE ORIENTE,"DOMICILIO: RETORNO 2 Y RIFLEROS SLP NO 18, U H...","-99.0368889998984,19.38020299985893,0",19.380203,-99.036889,RETORNO 2 Y RIFLEROS SLP NO 18,U HAB EJERCITO DE OTE II ISSSTE,IZTAPALAP,MEXICO DF.,NaN,NaN
4,PRIMARIA,GENERAL,NaN,ENRIQUE GARCIA GALLEGOS,"DOMICILIO: LATERAL RIO CHURUBUSCO NO 2, SAN J...","-99.11412044387679,19.3699792020639,0",19.369979,-99.114120,LATERAL RIO CHURUBUSCO NO 2,SAN JOSE ACULCO,IZTAPALAP,MEXICO DF.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,PRIMARIA,GENERAL,NaN,IGNACIO MARISCAL,"DOMICILIO: AGRUPAMIENTO F 101, U SAN JUAN DE A...","-99.056418000091,19.45569799983959,0",19.455698,-99.056418,AGRUPAMIENTO F 101,U SAN JUAN DE ARAGON,GUSTAVO A. MADER,MEXICO DF.,NaN,NaN
2379,SECUNDARIA,GENERAL,NaN,MIGUEL DE CERVANTES SAAVEDRA,"ANAXAGORAS NO 324, NARVARTE.","-99.1568090002494,19.39651199998512,0",19.396512,-99.156809,NaN,NaN,NaN,NaN,NaN,NaN
2380,PREESCOLAR,GENERAL,NaN,IRENE MARIN DE AYALA,"DOMICILIO: HUICHAPAN S/N, MICHOACANA, DELEGACI...","-99.11172663949883,19.44757589685136,0",19.447576,-99.111727,HUICHAPAN S/N,MICHOACANA,VENUSTIANO CARRANZ,MEXICO DF.,NaN,NaN
2381,SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,COY,MEXICO DF.,NaN,NaN


In [ ]:
def clean_text(text):
  text = text.remove(r"DELEGACION", r'', text)
  text = ' '.join(text.split())
  return text

publicas['delegaciones'] = publicas['Delegacion'].apply(lambda x: clean_text(x))


In [ ]:
drop_wrd = ['DELEGACION']
strings = publicas.Delegacion[:]
strings = ''.join((filter(lambda i: i not in drop_wrd, strings)))
print(strings)

TypeError: ignored

In [ ]:
publicas.delegaciones = []

for i in publicas.Delegacion:
  delegaciones = str(publicas.Delegacion)
  words = delegaciones.split()
  if i in words == "DELEGACION":
    drop = words.remove('DELEGACION')
    redo = ''.join(drop)
    publicas.delegaciones.append(redo)

publicas


,nivel,nivel_2,nivel_3,nombre,domicilio_c,coordenadas,latitud,longitud,domicilio,Unnamed: 9,Delegacion,Ciudad,Unnamed: 12,Unnamed: 13
0,INTERVENCION,"TEMPRANA,","PREESCOLAR,",CENTRO DE ATENCION MULTIPLE 82,DOMICILIO: AV TEZONTLE S/N ESQ ANDADOR AGAV...,"-99.10185339944889,19.38575798109795,0",19.385758,-99.101853,AV TEZONTLE S/N ESQ ANDADOR AGAVE,UNIDAD INFONAVIT IZTACALCO,DELEGACION IZTACALCO,MEXICO DF.,NaN,NaN
1,PRIMARIA,GENERAL,NaN,PROFRA. MARIA EPIGMENIA ARRIAGA SALGADO,"DOMICILIO: 2O RET ROSARIO CASTELLANOS MZA L, U...","-99.10427446479174,19.30970837543437,0",19.309708,-99.104274,2O RET ROSARIO CASTELLANOS MZA L,UNIDAD HABITACIONAL C T M,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN
2,PREESCOLAR,GENERAL,NaN,TINEMI,"DOMICILIO: JACARANDAS Y SABINO, BOSQUE DEL PED...","-99.2374303425192,19.27159235767396,0",19.271592,-99.237430,JACARANDAS Y SABINO,BOSQUE DEL PEDREGAL,DELEGACION TLAHUAC,MEXICO DF.,NaN,NaN
3,PREESCOLAR,GENERAL,CON,EJERCITO DE ORIENTE,"DOMICILIO: RETORNO 2 Y RIFLEROS SLP NO 18, U H...","-99.0368889998984,19.38020299985893,0",19.380203,-99.036889,RETORNO 2 Y RIFLEROS SLP NO 18,U HAB EJERCITO DE OTE II ISSSTE,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
4,PRIMARIA,GENERAL,NaN,ENRIQUE GARCIA GALLEGOS,"DOMICILIO: LATERAL RIO CHURUBUSCO NO 2, SAN J...","-99.11412044387679,19.3699792020639,0",19.369979,-99.114120,LATERAL RIO CHURUBUSCO NO 2,SAN JOSE ACULCO,DELEGACION IZTAPALAPA,MEXICO DF.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,PRIMARIA,GENERAL,NaN,IGNACIO MARISCAL,"DOMICILIO: AGRUPAMIENTO F 101, U SAN JUAN DE A...","-99.056418000091,19.45569799983959,0",19.455698,-99.056418,AGRUPAMIENTO F 101,U SAN JUAN DE ARAGON,DELEGACION GUSTAVO A. MADERO,MEXICO DF.,NaN,NaN
2379,SECUNDARIA,GENERAL,NaN,MIGUEL DE CERVANTES SAAVEDRA,"ANAXAGORAS NO 324, NARVARTE.","-99.1568090002494,19.39651199998512,0",19.396512,-99.156809,NaN,NaN,NaN,NaN,NaN,NaN
2380,PREESCOLAR,GENERAL,NaN,IRENE MARIN DE AYALA,"DOMICILIO: HUICHAPAN S/N, MICHOACANA, DELEGACI...","-99.11172663949883,19.44757589685136,0",19.447576,-99.111727,HUICHAPAN S/N,MICHOACANA,DELEGACION VENUSTIANO CARRANZA,MEXICO DF.,NaN,NaN
2381,SECUNDARIA,GENERAL,NaN,VICENTE GUERRERO,"DOMICILIO: CORINA NO 95, EL CARMEN, DELEGACION...","-99.15522099873616,19.35637199721737,0",19.356372,-99.155221,CORINA NO 95,EL CARMEN,DELEGACION COYOACAN,MEXICO DF.,NaN,NaN
